In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.6 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [3]:
import cv2
import os
from ultralytics import YOLO
from IPython.display import Video, display


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
model = YOLO('yolov8n.pt')

In [9]:
INPUT_VIDEO = "/content/drive/MyDrive/walking video fottage/854100-hd_1920_1080_25fps.mp4"
OUTPUT_PATH = "/content/output_gender_tracking.mp4"
COMPRESSED_PATH = "/content/output_compressed.mp4" # Required for Colab display

In [10]:
yolo = YOLO("yolov8n.pt")  # nano = low RAM


In [26]:
cap = cv2.VideoCapture(INPUT_VIDEO)

if not cap.isOpened():
    print(f"Error: Could not open video file {INPUT_VIDEO}")
else:
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # We are resizing to 640x360 as per your original code
    TARGET_WIDTH = 640
    TARGET_HEIGHT = 360

    # Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (TARGET_WIDTH, TARGET_HEIGHT))

    print("Processing video... this may take a moment.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize frame to match TARGET dimensions
        frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))

        # TRACKING
        results = model.track(
            frame,
            persist=True,
            classes=0, # 0 = person
            tracker="bytetrack.yaml",
            verbose=False
        )

        # DRAWING
        result = results[0]
        current_count = 0  # 1. Initialize count to 0 for this frame

        if result.boxes.id is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            ids = result.boxes.id.cpu().numpy()
            current_count = len(ids)

            for box, track_id in zip(boxes, ids):
                x1, y1, x2, y2 = map(int, box)

                # Draw Box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Draw ID
                cv2.putText(frame, f"ID: {int(track_id)}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 3. Draw the current frame count instead of total unique count
        cv2.putText(frame, f"Count: {current_count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print("Processing complete.")

Processing video... this may take a moment.
Processing complete.


In [28]:
if os.path.exists(OUTPUT_PATH):
    print("Converting video for browser playback...")
    os.system(f"ffmpeg -y -i {OUTPUT_PATH} -vcodec libx264 {COMPRESSED_PATH}")
    display(Video(COMPRESSED_PATH, embed=True, width=640))
else:
    print("Error: Output file was not created.")

Converting video for browser playback...
